In [233]:
from django.http import HttpResponse
from django.template import loader
from django.core.mail import send_mail
from django.shortcuts import render

from google_play_scraper import app
from google_play_scraper import search
from google_play_scraper import permissions
from urllib.parse import urlparse
from urllib.parse import parse_qs

from textblob import TextBlob
import re
import unidecode
from string import digits
import requests

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
import pickle

In [234]:
with open('apps.txt') as f:
    apps = f.read().splitlines()

In [235]:
title = []
description = []
score = []
rating = []
reviews = []
email = []
domain = []

contacts = []
location = []
storage = []
microphone = []
sms = []
wifi = []
deviceid = []
photos = []
phone = []
camera = []

other = []

for id in apps:
    result = app(
        id,
        lang='en',  # parse_qs(parsed_url.query)['hl'][0],
        country='in'  # parse_qs(parsed_url.query)['gl'][0]
    )
    title.append(result['title'])
    description.append(result['description'])
    rating.append(result['ratings'])
    em = result['developerEmail']
    domain.append(em[em.index('@') + 1:])
    email.append(em[0: em.index('@')])
    
    rr = result['comments']
    rr_string = ' '.join(word for word in rr)
    reviews.append(rr_string)
    score.append(result['score'])
    permit = permissions(
        id,
        lang='en',  # defaults to 'en'
        country='in',  # defaults to 'us'
    )  # dictionary
    permisions_list = list(permit.keys())

    if ("Location" in permisions_list):
        location.append(1)
    else:
        location.append(0)

    if ("Storage" in permisions_list):
        storage.append(1)
    else:
        storage.append(0)

    if ("Contacts" in permisions_list):
        contacts.append(1)
    else:
        contacts.append(0)

    if ("SMS" in permisions_list):
        sms.append(1)
    else:
        sms.append(0)
    
    if ("Microphone" in permisions_list):
        microphone.append(1)
    else:
        microphone.append(0)

    if ("Phone" in permisions_list):
        phone.append(1)
    else:
        phone.append(0)

    if ("Wi-Fi connection information" in permisions_list):
        wifi.append(1)
    else:
        wifi.append(0)

    if ("Camera" in permisions_list):
        camera.append(1)
    else:
        camera.append(0)

    if ("Photos/Media/Files" in permisions_list):
        photos.append(1)
    else:
        photos.append(0)

    if ("Device ID & call information" in permisions_list):
        deviceid.append(1)
    else:
        deviceid.append(0)
    a1 = []
    if('Other' in permisions_list):
        a1 = permit['Other']
    a2 = []
    if('Uncategorized' in permisions_list):
        a2 = permit['Uncategorized']
    a3 = a1 + a2
    a3_string = ' '.join(word for word in a3) 
    other.append(a3_string)

d = {'title': title, 'description': description, 'score': score, 'rating': rating, 'review': reviews, 'email': email, 'domain': domain, 'contacts': contacts, 'location': location, 'storage': storage, 'microphone': microphone, 'sms': sms, 'wifi': wifi, 'deviceid': deviceid, 'photos': photos, 'phone': phone, 'camera': camera, 'other': other}

df = pd.DataFrame(d)

In [236]:
# df.to_csv("myfile.csv")

In [241]:
df = pd.read_csv("myfile.csv")

train1 = df['description']
train2 = df['review']
train3 = df['other']
train4 = df['domain']
train5 = df['email']

k1 = len(train5)
k1

54

In [242]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2), min_df=1, max_df=1,
    max_features=200
)

word_vectorizer.fit(train1)
f1 = word_vectorizer.transform(train1)
word_vectorizer.fit(train2)
f2 = word_vectorizer.transform(train2)
word_vectorizer.fit(train3)
f3 = word_vectorizer.transform(train3)
word_vectorizer.fit(train4)
f4 = word_vectorizer.transform(train4)
word_vectorizer.fit(train5)
f5 = word_vectorizer.transform(train5)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6), min_df=1, max_df=1,
    max_features=200
)

char_vectorizer.fit(train1)
c1 = char_vectorizer.transform(train1)
char_vectorizer.fit(train2)
c2 = char_vectorizer.transform(train2)
char_vectorizer.fit(train3)
c3 = char_vectorizer.transform(train3)
char_vectorizer.fit(train4)
c4 = char_vectorizer.transform(train4)
char_vectorizer.fit(train5)
c5 = char_vectorizer.transform(train5)


c:\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(


In [243]:
a1 = np.array(df['score']).reshape(k1,1)
a2 = np.array(df['rating']).reshape(k1,1)
a3 = np.array(df['location']).reshape(k1,1)
a4 = np.array(df['storage']).reshape(k1,1)
a5 = np.array(df['microphone']).reshape(k1,1)
a6 = np.array(df['sms']).reshape(k1,1)
a7 = np.array(df['wifi']).reshape(k1,1)
a8 = np.array(df['deviceid']).reshape(k1,1)
a9 = np.array(df['photos']).reshape(k1,1)
a10 = np.array(df['phone']).reshape(k1,1)
a11 = np.array(df['camera']).reshape(k1,1)
a12 = np.array(df['contacts']).reshape(k1,1)

train_features = hstack([f1,f2,f3,f4,f5,c1,c2,c3,c4,c5,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12])
train_features.shape

(54, 1572)

In [244]:

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
# from sklearn.pipeline import make_union
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.feature_selection import SelectFromModel
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Ridge
from itertools import product
from sklearn.ensemble import VotingClassifier
# from sklearn.naive_bayes import ComplementNB
from scipy.sparse import csr_matrix
from imblearn.ensemble import EasyEnsembleClassifier

import lightgbm as lgb

from scipy.sparse import hstack
import regex as re
import re as r
import regex, string
import time
start = time.time()

scores = []
ridge_file = 'ridge.pckl'
ridge_model_pkl = open(ridge_file, 'wb')
train_target = df['fraud']

classifier = LogisticRegression(random_state=0)
# classifier = Ridge(alpha=29, copy_X=True, fit_intercept=True, solver='sag',
#                    max_iter=150, random_state=0,  tol=0.0025)

# cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3,
#                                    scoring='roc_auc'))
# scores.append(cv_score)

l = EasyEnsembleClassifier(base_estimator=LogisticRegression(C=2, solver='sag', max_iter=500))
n = EasyEnsembleClassifier(base_estimator=SGDClassifier(alpha=.0002, max_iter=180, penalty="l2", loss='modified_huber'))
o = LogisticRegression(C=2, dual=True, max_iter=500)
p = RandomForestClassifier(criterion='gini',
    max_depth=100, max_features=1000, max_leaf_nodes=None, min_samples_split=10,
    min_weight_fraction_leaf=0.0, n_estimators=120)  
#     r = Ridge(alpha=29, copy_X=True, fit_intercept=True, solver='sag',
#                      max_iter=500,   normalize=False, random_state=0,  tol=0.0025)
classifier.fit(train_features, train_target)
pickle.dump(classifier, ridge_model_pkl)

ridge_model_pkl.close()
# print('Total CV score is {}'.format(np.mean(scores)))
print(scores)


[]


In [245]:
df2 = pd.read_csv('myfile2.csv')

test1 = df2['description']
test2 = df2['review']
test3 = df2['other']
test4 = df2['domain']
test5 = df2['email']

k = len(test1)

word_vectorizer.fit(test1)
ft1 = word_vectorizer.transform(test1)
word_vectorizer.fit(test2)
ft2 = word_vectorizer.transform(test2)
word_vectorizer.fit(test3)
ft3 = word_vectorizer.transform(test3)
word_vectorizer.fit(test4)
ft4 = word_vectorizer.transform(test4)
word_vectorizer.fit(test5)
ft5 = word_vectorizer.transform(test5)


char_vectorizer.fit(test1)
ct1 = char_vectorizer.transform(test1)
char_vectorizer.fit(test2)
ct2 = char_vectorizer.transform(test2)
char_vectorizer.fit(test3)
ct3 = char_vectorizer.transform(test3)
char_vectorizer.fit(test4)
ct4 = char_vectorizer.transform(test4)
char_vectorizer.fit(test5)
ct5 = char_vectorizer.transform(test5)

at1 = np.array(df2['score']).reshape(k, 1)
at2 = np.array(df2['rating']).reshape(k, 1)
at3 = np.array(df2['location']).reshape(k, 1)
at4 = np.array(df2['storage']).reshape(k, 1)
at5 = np.array(df2['microphone']).reshape(k, 1)
at6 = np.array(df2['sms']).reshape(k, 1)
at7 = np.array(df2['wifi']).reshape(k, 1)
at8 = np.array(df2['deviceid']).reshape(k, 1)
at9 = np.array(df2['photos']).reshape(k, 1)
at10 = np.array(df2['phone']).reshape(k, 1)
at11 = np.array(df2['camera']).reshape(k, 1)
at12 = np.array(df2['contacts']).reshape(k, 1)

test_features = hstack([ft1, ft2, ft3, ft4, ft5, ct1, ct2, ct3, ct4, ct5, at1, at2, at3, at4, at5, at6, at7, at8, at9, at10, at11, at12])
test_features.shape

c:\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(


(2, 1177)

In [246]:
m = classifier.predict_proba(train_features)
m[0]
m[50]
m[44]

array([0.72747889, 0.27252111])